In [8]:
import random
%load_ext autoreload
%autoreload 2
import os
import json
import genesis_cloud.utils as u
from tqdm import tqdm
from step.step import *
import random
instance_url = '147.189.192.78:8080'


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
inputStep = InputStep('input text', '')
system_prompt ="""
<|im_start|>system
---------------------
```document.txt
{document}
```
---------------------
Given the above document, please answer whether or not a medication is cited in the document.
<|im_end|>
"""

params = json.load(open('params.json', 'r'))
params['grammar'] = 'root ::= "true" | "false"'


def map_input(inputs: list = None):
    return system_prompt.format(document = inputs[0].output.value)



initial_step = MapStep('add system prompt', map_input, [inputStep])
# step1 = Step(LLamaCppModel(instance_url), 'step1', prompt, params, map_input, inputs=[input_text])

In [10]:
prompt = """
{history}
<|im_start|>user
{row}
<|im_end|>
<|im_start|>assistant
"""

def get_history(inputs: list = None):
    history =( (str(inputs[0].input.value)) if inputs[0].input.value else '')  + inputs[0].output.value
    str_values = ' '.join([v for k,v in inputs[1].items() if k != 'line'])
    return {'history': history, 'row': str_values}

sample = json.load(open('sample.json', 'r'))
inputStep.input.value = sample['text']
ans = []
for _ in tqdm(range(5)):
    steps = [inputStep, initial_step]
    annotations = list(enumerate(sample['annotations']))
    random.shuffle(annotations)
    for i, annotation in annotations:
        step = Step(LLamaCppModel(instance_url), 'step' + str(i), prompt, params, get_history, inputs=[steps[-1], annotation])
        steps.append(step)
    pipeline = Pipeline(steps[-1])
    pipeline()
    
    
    full_chat = steps[-1].input.value + steps[-1].output.value
    answers = full_chat.split('<|im_start|>assistant\n')[1:]
    answers = [a.split('<|im_end|>')[0] for a in answers]
    answers = list(zip(list(map(lambda x: x[0], annotations)), answers))
    
    ans.append(answers)

100%|██████████| 5/5 [11:45<00:00, 141.13s/it]


In [11]:
res_s = [sorted(a, key=lambda x: x[0]) for a in ans]

In [20]:
res_ss = [list(map(lambda x: x[1], a)) for a in res_s]
res_ss = [[1 if x == 'true' else -1 for x in a] for a in res_ss]



In [22]:
# sum each row among columns
res_ss = [sum(x) for x in zip(*res_ss)]

In [23]:
res_ss

[5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 3,
 3,
 5,
 3,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 3,
 5,
 5,
 5,
 3,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 3,
 3,
 5,
 5,
 3,
 5,
 5,
 5,
 5,
 3,
 5,
 5,
 5,
 5,
 3]